In [1]:
import polars as pl

train = pl.read_csv("/home/shinichiro.saito/atmacup17/data/train_with_index.csv")

In [2]:
train

index,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count
i64,i64,i64,str,str,i64,i64,i64
0,0,25,"""3-season skirt!""","""Adorable, well-made skirt! lin…",5,1,4
1,0,39,"""Very cute""","""Love the asymmetrical hem. wai…",5,1,0
2,0,42,"""Beautiful! fruns small for typ…","""I love this skirt! i wasn't su…",5,1,5
3,0,45,null,"""I was really pleased with this…",5,1,9
4,0,57,"""Unique, pretty asymmetric skir…","""I saw this skirt in retailer s…",5,1,1
…,…,…,…,…,…,…,…
9995,232,57,"""Runs big on top""",null,3,1,5
9996,232,58,null,"""I loved the dress, but just no…",1,1,5
9997,232,60,"""I was really disappointed""","""I was really hoping this dress…",2,0,7


In [3]:
import numpy as np
from sklearn.model_selection import StratifiedKFold

fold_arr = np.zeros(train.height)
sgkf = StratifiedKFold(n_splits=3, random_state=42, shuffle=True)

for idx, (_, val_idx) in enumerate(sgkf.split(train, train["Recommended IND"])):
    fold_arr[val_idx] = idx

train = train.with_columns(pl.Series(fold_arr).cast(pl.Int64).alias("fold"))

In [4]:
# idとlabelをdictにして保存する
id_fold_dict = dict(zip(train["index"], train["fold"]))

In [5]:
# jsonで保存する
import json

with open(f"/home/shinichiro.saito/atmacup17/data/rec_stratified_fold.json", "w") as f:
    json.dump(id_fold_dict, f)

In [6]:
id_fold_dict

{0: 2,
 1: 2,
 2: 1,
 3: 0,
 4: 1,
 5: 0,
 6: 0,
 7: 2,
 8: 1,
 9: 2,
 10: 2,
 11: 1,
 12: 2,
 13: 1,
 14: 2,
 15: 2,
 16: 2,
 17: 2,
 18: 0,
 19: 1,
 20: 0,
 21: 2,
 22: 0,
 23: 2,
 24: 0,
 25: 1,
 26: 1,
 27: 1,
 28: 0,
 29: 1,
 30: 2,
 31: 0,
 32: 0,
 33: 2,
 34: 2,
 35: 1,
 36: 2,
 37: 0,
 38: 0,
 39: 2,
 40: 2,
 41: 2,
 42: 2,
 43: 2,
 44: 2,
 45: 1,
 46: 2,
 47: 2,
 48: 2,
 49: 1,
 50: 0,
 51: 2,
 52: 1,
 53: 2,
 54: 1,
 55: 2,
 56: 2,
 57: 2,
 58: 2,
 59: 1,
 60: 2,
 61: 1,
 62: 1,
 63: 0,
 64: 1,
 65: 0,
 66: 1,
 67: 0,
 68: 2,
 69: 0,
 70: 2,
 71: 1,
 72: 0,
 73: 2,
 74: 1,
 75: 1,
 76: 1,
 77: 2,
 78: 1,
 79: 2,
 80: 0,
 81: 2,
 82: 1,
 83: 0,
 84: 1,
 85: 2,
 86: 1,
 87: 1,
 88: 2,
 89: 1,
 90: 1,
 91: 1,
 92: 1,
 93: 1,
 94: 2,
 95: 2,
 96: 0,
 97: 1,
 98: 0,
 99: 1,
 100: 2,
 101: 0,
 102: 1,
 103: 0,
 104: 1,
 105: 0,
 106: 2,
 107: 1,
 108: 1,
 109: 0,
 110: 1,
 111: 0,
 112: 0,
 113: 0,
 114: 0,
 115: 2,
 116: 0,
 117: 2,
 118: 0,
 119: 1,
 120: 2,
 121: 2,
 122: 0,
 12